# 1. Preparación de Entorno

In [1]:
 dotenv import load_dotenv
import os

# Carga las variables de entorno desde el archivo .env
load_dotenv()

bucket = "datasetsbdajac"

#Utilitario para manifrompular los servicios de AWS
import boto3

In [2]:
access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
region = os.getenv('AWS_DEFAULT_REGION')

# 2. Conexión al servicio

In [4]:
#Obtenemos el cliente de servicio
rekognition = boto3.client(
  "rekognition", #Servicio al que nos conectamos
  aws_access_key_id = access_key_id, #Identificador de la clave
  aws_secret_access_key = secret_access_key, #Contraseña de la clave
  region_name = region #Región de la clave
)

# 3. Envío de consulta

In [5]:
#Enviamos la consulta
#Es una consulta asíncrona
respuesta = rekognition.detect_labels(
    Image = { #Indicamos la imagen
        "S3Object": { #Físicamente, la imagen se encuentra en S3
            "Bucket": bucket, #Bucket de la imagen, CAMBIAR "XXX" por nuestras iniciales
            "Name": "imagenes/IMAGEN_2.jpg" #Ruta de la imagen
        }
    },
    MaxLabels = 10 #Cantidad máxima de entidades que queremos detectar
)

# 4. Análisis de la respuesta

In [6]:
#Las entidades reconocidas se encuentran en la variable "Labels"
#Analicemos la primera entidad encontrada
#En la variable "Name" vemos la entidad detectada
#En la variable "Instances" si son OBJETOS o si son DESCRIPCIONES
respuesta["Labels"][0]

{'Name': 'Photography',
 'Confidence': 99.94345092773438,
 'Instances': [],
 'Parents': [],
 'Aliases': [{'Name': 'Photo'}],
 'Categories': [{'Name': 'Hobbies and Interests'}]}

In [7]:
#Analicemos la segunda entidad encontrada
#En la variable "Name" vemos la entidad detectada
#En la variable "Instances" si son OBJETOS o si son DESCRIPCIONES
respuesta["Labels"][1]

{'Name': 'Person',
 'Confidence': 97.91618347167969,
 'Instances': [{'BoundingBox': {'Width': 0.11297222971916199,
    'Height': 0.7152727246284485,
    'Left': 0.4107826054096222,
    'Top': 0.26713359355926514},
   'Confidence': 96.00708770751953},
  {'BoundingBox': {'Width': 0.16902107000350952,
    'Height': 0.8017257452011108,
    'Left': 0.4894786477088928,
    'Top': 0.16630369424819946},
   'Confidence': 89.83811950683594},
  {'BoundingBox': {'Width': 0.12308807671070099,
    'Height': 0.6766061782836914,
    'Left': 0.20343667268753052,
    'Top': 0.323275089263916},
   'Confidence': 89.00283813476562}],
 'Parents': [],
 'Aliases': [{'Name': 'Human'}],
 'Categories': [{'Name': 'Person Description'}]}

In [8]:
#Iteremos cada entidad detectada
for analisis in respuesta["Labels"]:

  #Descripciones
  if len(analisis["Instances"]) == 0:
    print("Descripcion encontrada: "+analisis["Name"])

  #Objetos
  else:
    print("Objeto encontrado:"+analisis["Name"])

Descripcion encontrada: Photography
Objeto encontrado:Person
Descripcion encontrada: Walking
Descripcion encontrada: Park
Objeto encontrado:Female
Objeto encontrado:Girl
Objeto encontrado:Teen
Objeto encontrado:Jeans
Descripcion encontrada: Pants
Objeto encontrado:Dog


# 5. Modelamiento de las descripciones

In [9]:
#Acumulador de lineas
descripciones = ""

In [10]:
#Iteramos cada línea
for analisis in respuesta["Labels"]:

  #Filtramos sólo las descripciones
  if len(analisis["Instances"]) == 0:

    #Definimos la linea
    linea = (
        str(analisis["Name"]) + "|" + #Descripción
        str(analisis["Confidence"]) #Probabilidad
    )

    #Imprimimos la línea
    print(linea)

    #Agregamos la línea y un salto de línea
    descripciones = descripciones + linea + "\n"

Photography|99.94345092773438
Walking|97.91618347167969
Park|96.988037109375
Pants|95.84716796875


In [11]:
#Agregamos una cabecera
descripciones = "Descripcion|Probabilidad" + "\n" + descripciones

#Verificamos
print(descripciones)

Descripcion|Probabilidad
Photography|99.94345092773438
Walking|97.91618347167969
Park|96.988037109375
Pants|95.84716796875



# 6. Análisis de los objetos detectados

In [12]:
#Acumulador de lineas
objetos = ""

In [13]:
#Iteramos cada línea
for analisis in respuesta["Labels"]:

  #Filtramos sólo los objetos
  if len(analisis["Instances"]) > 0:

    #Iteramos cada objeto
    for objeto in analisis["Instances"]:

      #Analizamos cada objeto
      #Cada objeto tiene su "BoundingBox" (rectángulo) y la probabilidad
      #El problema es que los "BoundingBox" están escalados entre 0 y 1
      print(objeto)

{'BoundingBox': {'Width': 0.11297222971916199, 'Height': 0.7152727246284485, 'Left': 0.4107826054096222, 'Top': 0.26713359355926514}, 'Confidence': 96.00708770751953}
{'BoundingBox': {'Width': 0.16902107000350952, 'Height': 0.8017257452011108, 'Left': 0.4894786477088928, 'Top': 0.16630369424819946}, 'Confidence': 89.83811950683594}
{'BoundingBox': {'Width': 0.12308807671070099, 'Height': 0.6766061782836914, 'Left': 0.20343667268753052, 'Top': 0.323275089263916}, 'Confidence': 89.00283813476562}
{'BoundingBox': {'Width': 0.11297222971916199, 'Height': 0.7152727246284485, 'Left': 0.4107826054096222, 'Top': 0.26713359355926514}, 'Confidence': 96.00708770751953}
{'BoundingBox': {'Width': 0.12308807671070099, 'Height': 0.6766061782836914, 'Left': 0.20343667268753052, 'Top': 0.323275089263916}, 'Confidence': 89.00283813476562}
{'BoundingBox': {'Width': 0.11297222971916199, 'Height': 0.7152727246284485, 'Left': 0.4107826054096222, 'Top': 0.26713359355926514}, 'Confidence': 96.00708770751953}


# 7. Des-normalización de los Bounding Box

In [15]:
#Obtenemos el cliente de servicio
s3 = boto3.client(
  "s3", #Servicio al que nos conectamos
  aws_access_key_id = access_key_id, #Identificador de la clave
  aws_secret_access_key = secret_access_key, #Contraseña de la clave
  region_name = region #Región de la clave
)

In [16]:
#Obtenemos la imagen desde S3
archivo = s3.get_object(
  Bucket = bucket,
  Key = "imagenes/IMAGEN_2.jpg"
)

In [17]:
#Leemos la imagen binarizada
contenidoBinarioDeImagen = archivo["Body"].read()

In [18]:
#Utilitario para cargar contenido binario
from io import BytesIO

In [19]:
#Utilitario para recibir contenido binario proveniente de una imagen
from PIL import Image

In [20]:
#Leemos la imagen
imagen = Image.open(BytesIO(contenidoBinarioDeImagen))

In [21]:
#Obtiene las dimensiones de la imagen
width, height = imagen.size

In [22]:
#Verificamos
width, height

(1380, 920)

In [23]:
#Iteramos cada línea
for analisis in respuesta["Labels"]:

  #Filtramos sólo los objetos
  if len(analisis["Instances"]) > 0:

    #Iteramos cada objeto
    for objeto in analisis["Instances"]:
      #Des-normalizamos
      x = int(objeto["BoundingBox"]["Left"] * width)
      y = int(objeto["BoundingBox"]["Top"] * height)
      w = int(objeto["BoundingBox"]["Width"] * width)
      h = int(objeto["BoundingBox"]["Height"] * height)

      #Definimos la línea
      linea = (
          str(x) + "|" +
          str(y) + "|" +
          str(w) + "|" +
          str(h)
      )

      #Verificamos
      print(linea)

566|245|155|658
675|152|233|737
280|297|169|622
566|245|155|658
280|297|169|622
566|245|155|658
566|245|155|658
585|495|134|398
755|647|265|271


In [24]:
#Acumulador de lineas
objetos = ""

In [25]:
#Iteramos cada línea
for analisis in respuesta["Labels"]:

  #Filtramos sólo los objetos
  if len(analisis["Instances"]) > 0:

    #Iteramos cada objeto
    for objeto in analisis["Instances"]:
      #Des-normalizamos
      x = int(objeto["BoundingBox"]["Left"] * width)
      y = int(objeto["BoundingBox"]["Top"] * height)
      w = int(objeto["BoundingBox"]["Width"] * width)
      h = int(objeto["BoundingBox"]["Height"] * height)

      #Definimos la línea
      linea = (
          str(analisis["Name"]) + "|" +
          str(objeto["Confidence"]) + "|" +
          str(x) + "|" +
          str(y) + "|" +
          str(w) + "|" +
          str(h)
      )

      #Verificamos
      print(linea)

      #Agregamos la línea y un salto de línea
      objetos = objetos + linea + "\n"

Person|96.00708770751953|566|245|155|658
Person|89.83811950683594|675|152|233|737
Person|89.00283813476562|280|297|169|622
Female|96.00708770751953|566|245|155|658
Female|89.00283813476562|280|297|169|622
Girl|96.00708770751953|566|245|155|658
Teen|96.00708770751953|566|245|155|658
Jeans|95.84716796875|585|495|134|398
Dog|94.00374603271484|755|647|265|271


In [26]:
#Agregamos una cabecera
objetos = "Objeto|Probabilidad|x|y|w|h" + "\n" + objetos

#Verificamos
print(objetos)

Objeto|Probabilidad|x|y|w|h
Person|96.00708770751953|566|245|155|658
Person|89.83811950683594|675|152|233|737
Person|89.00283813476562|280|297|169|622
Female|96.00708770751953|566|245|155|658
Female|89.00283813476562|280|297|169|622
Girl|96.00708770751953|566|245|155|658
Teen|96.00708770751953|566|245|155|658
Jeans|95.84716796875|585|495|134|398
Dog|94.00374603271484|755|647|265|271

